# Week 4 Notebook


```
Level 1: Query Classification

Your first task is to generate training data that fastText can learn from. You’ve already worked with examples of parsing the category tree as XML, pruning the category tree to a maximum depth, and mapping the leaf category ids of queries to ancestor categories. Feel free to review these in the above reading materia!
```

In [ ]:
%%bash
pip install -r ../requirements_week4.txt

In [4]:
%%bash
python ../week4/create_labeled_queries.py

In [8]:
%%bash
head -n 10 /workspace/datasets/labeled_query_data.txt

__label__abcat0101001 Televisiones Panasonic  50 pulgadas
__label__abcat0101001 Sharp
__label__pcmcat193100050014 nook
__label__abcat0101001 rca
__label__abcat0101005 rca
__label__pcmcat143200050016 Flat screen tvs
__label__pcmcat247400050001 macbook
__label__pcmcat171900050028 Blue tooth headphones
__label__abcat0107004 Tv antenna
__label__pcmcat186100050006 memory card


## Step 1
- Work on cleaning queries / stemming etc

In [15]:
import pandas as pd
df = pd.read_csv('/workspace/datasets/train.csv')[['category', 'query']]
df.head()

,category,query
0,abcat0101001,Televisiones Panasonic 50 pulgadas
1,abcat0101001,Sharp
2,pcmcat193100050014,nook
3,abcat0101001,rca
4,abcat0101005,rca


In [58]:
import utilities.functions as fn

cleaned_queries = []

for index, row in df.head(20).iterrows():
    query = row["query"]
    #
    # Clean the queries using shared function
    #
    normalized_query = fn.clean_query(query)
    print('query="{0}" clean="{1}"'.format(query, normalized_query))
    #
    # Create new rows for cleaned queries
    #
    new_row = {};
    new_row["query"] = normalized_query;
    new_row["category"] = row["category"]
    clean_queries.append(new_row)
    
cleaned_queries_df = pd.DataFrame(clean_queries)
cleaned_queries_df.head()

query="Televisiones Panasonic  50 pulgadas" clean="television panason pulgada"
query="Sharp" clean="sharp"
query="nook" clean="nook"
query="rca" clean="rca"
query="rca" clean="rca"
query="Flat screen tvs" clean="flat screen tv"
query="macbook" clean="macbook"
query="Blue tooth headphones" clean="blue tooth headphon"
query="Tv antenna" clean="tv antenna"
query="memory card" clean="memori card"
query="AC power cord" clean="ac power cord"
query="Zagg iPhone" clean="zagg iphon"
query="Watch The Throne" clean="watch throne"
query="Remote control extender" clean="remot control extend"
query="Camcorder" clean="camcord"
query="3ds" clean="3ds"
query="hoya" clean="hoya"
query="wireless headphones" clean="wireless headphon"
query="wireless headphones" clean="wireless headphon"
query="Samsung 40" clean="samsung"


,query,category
0,television panason pulgada,abcat0101001
1,sharp,abcat0101001
2,nook,pcmcat193100050014
3,rca,abcat0101001
4,rca,abcat0101005


In [86]:
%%bash
python ../week4/create_labeled_queries.py
head -n 10 /workspace/datasets/labeled_query_data.txt
wc -l /workspace/datasets/labeled_query_data.txt

Traceback (most recent call last):
  File "/workspace/search_with_machine_learning_course/week4/../week4/create_labeled_queries.py", line 53, in <module>
    df = pd.read_csv(queries_file_name)[['category', 'query']]
  File "/home/gitpod/.pyenv/versions/search_with_ml_week4/lib/python3.9/site-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/home/gitpod/.pyenv/versions/search_with_ml_week4/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 680, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/gitpod/.pyenv/versions/search_with_ml_week4/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 581, in _read
    return parser.read(nrows)
  File "/home/gitpod/.pyenv/versions/search_with_ml_week4/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 1250, in read
    index, columns, col_dict = self._engine.read(nrows)
  File "/home/gitpod/.pyenv/versions/search_with_ml_week4/lib/python3.9/si

Error while terminating subprocess (pid=9358): 


In [82]:
import pandas as pd
import xml.etree.ElementTree as ET
categories_file_name = r'/workspace/datasets/product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml'
root_category_id = 'cat00000'

tree = ET.parse(categories_file_name)
root = tree.getroot()

# Parse the category XML file to map each category id to its parent category id in a dataframe.
categories = []
parents = []
for child in root:
    id = child.find('id').text
    cat_path = child.find('path')
    cat_path_ids = [cat.find('id').text for cat in cat_path]
    leaf_id = cat_path_ids[-1]
    if leaf_id != root_category_id:
        categories.append(leaf_id)
        parents.append(cat_path_ids[-2])
parents_df = pd.DataFrame(list(zip(categories, parents)), columns =['category', 'parent'])

parents_df.head()

,category,parent
0,abcat0010000,cat00000
1,abcat0011000,abcat0010000
2,abcat0011001,abcat0011000
3,abcat0011002,abcat0011000
4,abcat0011003,abcat0011000


In [98]:
def get_category_size(frame, category):
    return frame[frame['category'] == category].size;

def get_parent_category(frame, category):
    return frame[frame['category'] == category]['parent'].item();

for cat in test_categories:
    print("cat:{0}, size:{1}, parent:{2}".format(cat, get_category_size(df, cat), get_parent_category(parents_df, cat)))

cat:pcmcat143200050016, size:2700, parent:pcmcat139900050002
cat:abcat0101005, size:2084, parent:abcat0101000
